In [ ]:
import re
import urllib.request
import json
import numpy as np
import pandas as pd
import re

#Read Data in From DraftKings
url = "https://sportsbook.draftkings.com/api/odds/v1/leagues/3/offers/gamelines.json"
request = urllib.request.Request(url)
page = urllib.request.urlopen(request).read()
b = json.loads(page)
b['errorStatus']


In [ ]:
url = "https://sportsbook.draftkings.com/api/odds/v1/leagues/"
request = urllib.request.Request(url)

page = urllib.request.urlopen(request).read()
b = json.loads(page)

league_df = pd.json_normalize(b, record_path =['leagues'])
print(league_df.head())

In [ ]:
#Read Data in From DraftKings
url = "https://sportsbook.draftkings.com/api/odds/v1/leagues/3/offers/gamelines.json"
request = urllib.request.Request(url)
page = urllib.request.urlopen(request).read()
b = json.loads(page)
b['errorStatus']
'errorStatus' in b.keys()

In [ ]:
ct = 0
print(len(league_df[league_df['sportName']=='Soccer']))

for i in [88670607]:# league_df.loc[league_df['sportName']=='Soccer', 'leagueId']:
    url = f"https://sportsbook.draftkings.com/api/odds/v1/leagues/{i}/offers/gamelines.json"
    request = urllib.request.Request(url)
    page = urllib.request.urlopen(request).read()
    b = json.loads(page)
    if 'errorStatus' in b.keys():
        ct += 1
b = json.loads(page)

In [ ]:
raw_data = pd.json_normalize(data=b['events'], meta=['name', 'startDate', ['offers','id'], ['offers','label']],record_path=['offers', 'outcomes'])
data = raw_data

In [ ]:
# Group bet ID
data['same_bet'] = data['id'].apply(lambda x:''.join(re.split('P|N', x.split('_')[-2])))
# Mathematically Sensible Odds
data['odd_math'] = data['oddsDecimal'] - 1.0
# Seed Fields
data['bet1'] = 100.0 
data['bet2_30boost_on_pos'] = np.nan
data['profit_1'] = np.nan
data['profit_2'] = np.nan
data['good_bet'] = False
data['min_boost_to_positive'] = np.nan
data['min_boost_to_negitive'] = np.nan

Profit Must be greater than 0 on both sides  
$B_1 + B_1O_1 -B_1- B_2 >= 0$  
$B_2 +B_2O_2-B1-B2 >=0$

$B_1O_1 - B_2 = B_2O_2 -B_1$


$\large
B_2 = \frac{B_1(O_1 + 1)}{O_2 + 1}
$

Subject to $B_1 + B_1O_1 - B_2 >= 0$  

Sub in $B_2$

$\large
B_1O_1 >= \frac{B_1(O_1 + 1)}{O_2 + 1
}$

$\large
\frac{O_1}{O_1 + 1} >= \frac{1}{O_2 + 1}
$


What boost $x$ to $O_1$ will satistfy this inequality?

$\large
\frac{O_1 + x}{O_1 + x + 1} >= \frac{1}{O_2 + 1}
$


$\large
x = \frac{1}{O_2} - O_1
$




In [ ]:
# https://www.wolframalpha.com/input/?i=%28a%2Bx%29+%2F+%28a%2B1%2Bx%29+%3D+1+%2F%28b%2B1%29++for+x
for bet in data['same_bet'].unique():
    mask = data['same_bet'] == bet
    c = data[mask]
    if len(c) ==2:
        data.loc[mask,'min_boost_to_positive'] = 1/c['odd_math'].min() - c['odd_math'].max()
        data.loc[mask,'min_boost_to_negitive'] = 1/c['odd_math'].max() - c['odd_math'].min()

        data.loc[mask,'bet2_30boost_on_pos'] = c['bet1'].mean() * (c['odd_math'].max() + 0.30) / (c['odd_math'].min() + 1)
        data.loc[mask,'profit_1'] =  c['bet1'].mean() + c['bet1'].mean() + (c['odd_math'].max() + 0.30)
        data.loc[mask,'profit_2'] =  data.loc[mask,'bet2_30boost_on_pos'] + data.loc[mask,'bet2_30boost_on_pos'] + (c['odd_math'].min())

In [ ]:
# Assume 30 Percent Boost



data1 = data[abs(data['min_boost_to_positive'])<1]
data1.to_csv('sample.csv')

# how much you wiin - wagers
# Group on Line 
# Bet 100 on Plus 100
2-0ML62741785_1 - win
2-0ML62741785_2 draw (not wanted)
2-0ML62741785_3 - loss
